In [26]:
## >> Tut : http://deeplearning.net/tutorial/logreg.html <<<

import cPickle
import gzip
import os
import sys
import timeit

import numpy as np

import theano
import theano.tensor as T


In [63]:
## Load Data ##
def load_data(dataset):
    data_dir, data_file = os.path.split(dataset)
    if data_dir == "" and not os.path.isfile(dataset):
        new_path = os.path.join(
            os.path.split(__file__)[0],
            "..",
            "data",
            dataset
        )
        if os.path.isfile(new_path) or data_file == 'mnist.pkl.gz':
            dataset = new_path

    if (not os.path.isfile(dataset)) and data_file == 'mnist.pkl.gz':
        import urllib
        origin = (
            'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        )
        print 'Downloading data from %s' % origin
        urllib.urlretrieve(origin, dataset)

    print '... loading data'

    f = gzip.open(dataset, 'rb')
    train_set, valid_set, test_set = cPickle.load(f)
    f.close()

    def shared_dataset(data_xy, borrow=True):
        data_x, data_y = data_xy
        shared_x = theano.shared(numpy.asarray(data_x,
                                               dtype=theano.config.floatX),
                                 borrow=borrow)
        shared_y = theano.shared(numpy.asarray(data_y,
                                               dtype=theano.config.floatX),
                                 borrow=borrow)
        return shared_x, T.cast(shared_y, 'int32')

    test_set_x, test_set_y = shared_dataset(test_set)
    valid_set_x, valid_set_y = shared_dataset(valid_set)
    train_set_x, train_set_y = shared_dataset(train_set)

    rval = [(train_set_x, train_set_y), (valid_set_x, valid_set_y),
            (test_set_x, test_set_y)]
    return rval

In [49]:
# Logistic Regression Class

class LogisticReg(object):
    def __init__(self, input, n_in, n_out):
        self.w = theano.shared(
            value=numpy.zeros(
                (n_in, n_out),
                dtype=theano.config.floatX
            ),
            name='w',
            borrow=True
        )
         # initialize the biases b as a vector of n_out 0s
        self.b = theano.shared(
            value=numpy.zeros(
                (n_out,),
                dtype=theano.config.floatX
            ),
            name='b',
            borrow=True
        )
        self.p_y_given_x = T.nnet.softmax(T.dot(input,self.w) + self.b)
        self.y_pred = np.argmax(self.p_y_given_x,axis = 1)
        self.params = [self.w, self.b]
        self.input = input

        
    def neg_log_likelihood(self,y):
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]), y])
    
    def errors(self,y):
        if y.ndim != self.y_pred.ndim:
            raise TypeError(
                'y should have the same shape as self.y_pred',
                ('y', y.type, 'y_pred', self.y_pred.type)
            )
        if y.dtype.startswith('int'):
            return T.mean(T.neq(self.y_pred, y))
        else:
            raise NotImplementedError()

In [96]:
## Instantiate the class

x = T.matrix('x')
y = T.ivector('y')

index = T.lscalar('index')

classifier = LogisticReg(input=x, n_in=28 * 28, n_out=10)

cost = classifier.neg_log_likelihood(y)

#classifier = Logistic(input=x,n_in=28*28,n_out=10)

In [97]:
#### Train symbolic function : Compile ####

# gradients
gw,gb = T.grad(cost,[classifier.w,classifier.b])

# Update weight and bias
updates=[(classifier.w, classifier.w - (0.13*gw) ),(classifier.b, classifier.b - (0.13*gb))]

# train function
train = theano.function(inputs=[index],
                        outputs=[cost],
                        updates = updates,
                        givens={x : train_set_x[index*batch_size : (index+1)*batch_size],
                                y : train_set_y[index*batch_size :(index+1)*batch_size]}                        
                       )

In [70]:
## Load dataset ##
datasets = load_data('mnist.pkl.gz')

train_set_x, train_set_y = datasets[0]
valid_set_x, valid_set_y = datasets[1]
test_set_x, test_set_y = datasets[2]

... loading data


In [98]:
print train_set_x.get_value().shape
print test_set_x.get_value().shape

# Set minibatch size
batch_size = 600

# compute number of minibatches for training, validation and testing
n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size


(50000, 784)
(10000, 784)


In [100]:
minibatch_avg_cost = 0
for j in xrange(100):
    for i in xrange(n_train_batches):
        minibatch_avg_cost = train(i)
    print 'iteration ',j,' : cost : ',minibatch_avg_cost

iteration  0  : cost :  [array(0.3216033906150585)]
iteration  1  : cost :  [array(0.32141156836623513)]
iteration  2  : cost :  [array(0.3212224491328717)]
iteration  3  : cost :  [array(0.32103596997239303)]
iteration  4  : cost :  [array(0.3208520700829759)]
iteration  5  : cost :  [array(0.32067069070752896)]
iteration  6  : cost :  [array(0.3204917750428766)]
iteration  7  : cost :  [array(0.32031526815381706)]
iteration  8  : cost :  [array(0.32014111689175667)]
iteration  9  : cost :  [array(0.319969269817638)]
iteration  10  : cost :  [array(0.3197996771289019)]
iteration  11  : cost :  [array(0.3196322905902374)]
iteration  12  : cost :  [array(0.319467063467896)]
iteration  13  : cost :  [array(0.3193039504673561)]
iteration  14  : cost :  [array(0.31914290767414183)]
iteration  15  : cost :  [array(0.318983892497611)]
iteration  16  : cost :  [array(0.3188268636175405)]
iteration  17  : cost :  [array(0.31867178093334736)]
iteration  18  : cost :  [array(0.31851860551579686)

KeyboardInterrupt: 

In [102]:
test = theano.function(inputs=[index],outputs=classifier.errors(y),
                      givens={x : test_set_x[index*batch_size : (index +1)*batch_size],
                              y : test_set_y[index*batch_size : (index +1)*batch_size]})

In [104]:
for j in xrange(n_test_batches):
    avg_test_error = test(j)
    print 'batch ',j,' : avg_error : ',avg_test_error
    

batch  0  : avg_error :  0.085
batch  1  : avg_error :  0.101666666667
batch  2  : avg_error :  0.115
batch  3  : avg_error :  0.106666666667
batch  4  : avg_error :  0.0916666666667
batch  5  : avg_error :  0.0716666666667
batch  6  : avg_error :  0.111666666667
batch  7  : avg_error :  0.0983333333333
batch  8  : avg_error :  0.0633333333333
batch  9  : avg_error :  0.0616666666667
batch  10  : avg_error :  0.0666666666667
batch  11  : avg_error :  0.0433333333333
batch  12  : avg_error :  0.02
batch  13  : avg_error :  0.0716666666667
batch  14  : avg_error :  0.0133333333333
batch  15  : avg_error :  0.0483333333333


In [ ]:
test = theano.function(inputs=[index],outputs=classifier.errors(y),
                      givens={x : valid_set_x[index*batch_size : (index +1)*batch_size],
                              y : valid_set_y[index*batch_size : (index +1)*batch_size]})

for j in xrange(n_valid_batches):
    avg_v_error = test(j)
    print 'batch ',j,' : avg_error : ',avg_valid_error
    